<h2>Preparation phase</h2>

Objective for training dataframe:
- Cleaning the data (missing data imputation/replacement, change the string to numeric, and so on)
- Split the data frame into subsets (if the size of the training data is very large) and perform feature extraction/feature engineering and then merge the result
- Split training dataframe into two part; 80% of the data for the training itself, and other 20% is for validation (repeat the process using different subset and it become cross-validation)

Objective for test dataframe:
- Cleaning the data (missing data imputation/replacement, change the string to numeric, and so on)
- Split the data frame into subsets (if the size of the training data is very large) and perform feature extraction/feature engineering and then merge the result. Create the same features for the test set

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('preparation.csv',delimiter=',')

In [3]:
df.shape

(59400, 19)

In [4]:
!ls

label.csv	      submission1.csv	 SubmissionFormat.csv	trainingset.csv
Preparatin_TzW.ipynb  submission2.csv	 Tanzanian Water.ipynb
preparation.csv       submission3rf.csv  testset.csv


#take 80% as train dataframe, and leave 20% as a validator/to check the accuracy of our prediction before applying the model to the real test data.

In [5]:
59400*0.8

47520.0

In [6]:
dftrain = pd.read_csv('preparation.csv',delimiter=',',nrows=47520)

In [7]:
dftrain.tail()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,water_quality,quality_group,quantity_group,source_type,source_class,waterpoint_type,status_group,wp_age,daysdiff
47515,34684,0.0,0,33.457381,-3.552051,0,17,7,0,0,2,2,1,4,1,3,1,2017,1655.0
47516,8766,0.0,1321,30.592280,-3.733459,0,16,1,130,2003,1,1,1,6,2,1,3,14,1529.0
47517,22662,0.0,1306,35.262273,-9.831219,0,10,2,0,1994,1,1,3,6,2,2,2,23,1512.0
47518,39854,50.0,-13,38.979195,-6.519993,0,6,1,60,1995,1,1,1,6,2,1,1,22,2216.0
47519,45078,0.0,0,33.420152,-2.795071,0,19,4,0,0,1,1,1,5,1,3,1,2017,2080.0


In [19]:
dftrain['terrain'] = 'dataran tinggi'

In [21]:
dftrain['terrain'][dftrain['gps_height']<=500] = 'dataran rendah'

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [22]:
dftrain['terrain'][(dftrain['gps_height']>500) & (dftrain['gps_height']<=1000)] = 'dataran sedang'

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [24]:
dftrain['terrain'].replace('dataran rendah',0,inplace=True)
dftrain['terrain'].replace('dataran sedang',1,inplace=True)
dftrain['terrain'].replace('dataran tinggi',2,inplace=True)

In [27]:
dftrain.head()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,...,quality_group,quantity_group,source_type,source_class,waterpoint_type,status_group,wp_age,daysdiff,terrain,water_availability
0,69572,6000.0,1390,34.938093,-9.856322,0,11,5,109,1999,...,1,1,1,1,1,1,18,2225.0,2,1
1,8776,0.0,1399,34.698766,-2.147466,0,20,2,280,2010,...,1,2,2,2,1,1,7,1502.0,2,0
2,34310,25.0,686,37.460664,-3.821329,0,21,4,250,2009,...,1,1,3,2,2,1,8,1511.0,1,1
3,67743,0.0,263,38.486161,-11.155298,0,90,63,58,1986,...,1,3,4,1,2,2,31,1539.0,0,0
4,19728,0.0,0,31.130847,-1.825359,0,18,1,0,0,...,1,4,2,2,1,1,2017,2104.0,0,0


In [28]:
print dftrain.columns

Index([u'id', u'amount_tsh', u'gps_height', u'longitude', u'latitude',
       u'num_private', u'region_code', u'district_code', u'population',
       u'construction_year', u'water_quality', u'quality_group',
       u'quantity_group', u'source_type', u'source_class', u'waterpoint_type',
       u'status_group', u'wp_age', u'daysdiff', u'terrain',
       u'water_availability'],
      dtype='object')


In [26]:
dftrain['water_availability'] = dftrain.apply(lambda row: 0 if row['amount_tsh']==0.0 else 1,axis=1)

<h3>#prepare the training dataset for export</h3>

In [29]:
dftrain2 = dftrain[['id','water_availability','terrain','num_private','region_code','district_code','population','water_quality','quality_group','quantity_group','source_type','source_class','waterpoint_type','wp_age','daysdiff','status_group']].copy()

In [30]:
dftrain2.head()

,id,water_availability,terrain,num_private,region_code,district_code,population,water_quality,quality_group,quantity_group,source_type,source_class,waterpoint_type,wp_age,daysdiff,status_group
0,69572,1,2,0,11,5,109,1,1,1,1,1,1,18,2225.0,1
1,8776,0,2,0,20,2,280,1,1,2,2,2,1,7,1502.0,1
2,34310,1,1,0,21,4,250,1,1,1,3,2,2,8,1511.0,1
3,67743,0,0,0,90,63,58,1,1,3,4,1,2,31,1539.0,2
4,19728,0,0,0,18,1,0,1,1,4,2,2,1,2017,2104.0,1


In [47]:
dftrain2.to_csv('dftrain.csv',sep=',',header=True,index=False)

<h3>#prepare the validation dataset</h3>

The validation dataset must have same table structure with the training dataset so that we can calculate the accuracy.

In [33]:
dfvalidation = pd.read_csv('preparation.csv',delimiter=',',skiprows=range(1,47520),nrows=11880)

In [38]:
print dfvalidation.shape
dfvalidation.tail()

(11880, 19)


,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,water_quality,quality_group,quantity_group,source_type,source_class,waterpoint_type,status_group,wp_age,daysdiff
11875,11164,500.0,351,37.634053,-6.124830,0,5,6,89,2007,1,1,1,4,1,1,2,10,2230.0
11876,60739,10.0,1210,37.169807,-3.253847,0,3,5,125,1999,1,1,1,1,1,1,1,18,1444.0
11877,27263,4700.0,1212,35.249991,-9.070629,0,11,4,56,1996,1,1,1,6,2,1,1,21,2171.0
11878,37057,0.0,0,34.017087,-8.750434,0,12,7,0,0,4,4,1,4,1,3,1,2017,2197.0
11879,31282,0.0,0,35.861315,-6.378573,0,1,4,0,0,1,1,2,5,1,3,1,2017,2231.0


In [39]:
dfvalidation['terrain'] = 'dataran tinggi'

In [40]:
dfvalidation['terrain'][dftrain['gps_height']<=500] = 'dataran rendah'
dfvalidation['terrain'][(dfvalidation['gps_height']>500) & (dfvalidation['gps_height']<=1000)] = 'dataran sedang'

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [41]:
dfvalidation['terrain'].replace('dataran rendah',0,inplace=True)
dfvalidation['terrain'].replace('dataran sedang',1,inplace=True)
dfvalidation['terrain'].replace('dataran tinggi',2,inplace=True)

In [42]:
dfvalidation.head()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,water_quality,quality_group,quantity_group,source_type,source_class,waterpoint_type,status_group,wp_age,daysdiff,terrain
0,45078,0.0,0,33.420152,-2.795071,0,19,4,0,0,1,1,1,5,1,3,1,2017,2080.0,2
1,31768,50.0,99,38.991021,-9.255090,0,80,13,860,1985,1,1,1,5,1,3,1,32,1546.0,2
2,36242,0.0,1518,37.314627,-3.218825,0,3,4,30,1976,1,1,1,1,1,0,3,41,1504.0,2
3,18510,1000.0,0,38.803419,-4.865751,0,4,8,40,1990,1,1,2,1,1,1,1,27,2223.0,0
4,45518,0.0,0,32.561960,-9.174393,0,12,6,0,0,1,1,1,6,2,1,1,2017,2108.0,0


In [43]:
dfvalidation['water_availability'] = dfvalidation.apply(lambda row: 0 if row['amount_tsh']==0.0 else 1,axis=1)

In [44]:
dfvalidation2 = dfvalidation[['id','water_availability','terrain','num_private','region_code','district_code','population','water_quality','quality_group','quantity_group','source_type','source_class','waterpoint_type','wp_age','daysdiff','status_group']].copy()

In [45]:
dfvalidation2.head()

,id,water_availability,terrain,num_private,region_code,district_code,population,water_quality,quality_group,quantity_group,source_type,source_class,waterpoint_type,wp_age,daysdiff,status_group
0,45078,0,2,0,19,4,0,1,1,1,5,1,3,2017,2080.0,1
1,31768,1,2,0,80,13,860,1,1,1,5,1,3,32,1546.0,1
2,36242,0,2,0,3,4,30,1,1,1,1,1,0,41,1504.0,3
3,18510,1,0,0,4,8,40,1,1,2,1,1,1,27,2223.0,1
4,45518,0,0,0,12,6,0,1,1,1,6,2,1,2017,2108.0,1


In [46]:
dfvalidation2.to_csv('validation.csv',sep=',',header=True,index=False)

In [48]:
!ls -l

total 34068
-rw-r--r-- 1 febrifahmi febrifahmi  2094264 Apr 16 06:12 dftrain.csv
-rw-r--r-- 1 febrifahmi febrifahmi  1148327 Apr 14 21:12 label.csv
-rw-r--r-- 1 febrifahmi febrifahmi    50583 Apr 16 06:12 Preparatin_TzW.ipynb
-rw-r--r-- 1 febrifahmi febrifahmi  4495565 Apr 16 04:49 preparation.csv
-rw-r--r-- 1 febrifahmi febrifahmi   443312 Apr 15 07:43 submission1.csv
-rw-r--r-- 1 febrifahmi febrifahmi   278500 Apr 15 07:55 submission2.csv
-rw-r--r-- 1 febrifahmi febrifahmi   278077 Apr 15 08:35 submission3rf.csv
-rw-r--r-- 1 febrifahmi febrifahmi   324512 Apr 14 21:14 SubmissionFormat.csv
-rw-r--r-- 1 febrifahmi febrifahmi   135699 Apr 16 04:51 Tanzanian Water.ipynb
-rw-r--r-- 1 febrifahmi febrifahmi  5016337 Apr 14 21:15 testset.csv
-rw-r--r-- 1 febrifahmi febrifahmi 20069199 Apr 14 20:59 trainingset.csv
-rw-r--r-- 1 febrifahmi febrifahmi   523733 Apr 16 06:12 validation.csv


In [49]:
print "done"

done
